In [128]:
# Import Pandas
import pandas as pd

# Import Numpy
import numpy as np

# Use the inline backend to generate the plots within the browser
%matplotlib inline 

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style("darkgrid")

font = {'size'   : 14}
mpl.rc('font', **font)

# Use Folium to display the Maps for Visualisation
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from folium import plugins

from folium import plugins
from folium.plugins import HeatMap


from sklearn.neighbors import DistanceMetric



In [156]:
# Very useful function to calculate the great circle distance between two points
# Found here and adapted:
# https://stackoverflow.com/questions/40452759/pandas-latitude-longitude-to-distance-between-successive-rows
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))

    km = 6367 * c
    return km


def haversine(row):
    dist = DistanceMetric.get_metric('haversine')
   
    lat1, lon1, lat2, lon2 = map(np.radians, [row['latitude'], 
                                              row['longitude'], 
                                              row['ven_lat'], 
                                              row['ven_lon']])

    X = [[lat1, lon1],
         [lat2, lon2]]
    
    kms = 6367
    
    return kms * dist.pairwise(X)[0][1]

In [138]:
# Import the Pickle of the Crimes DataFrame
df_crimes = pd.read_pickle('./capstone_pickles/crimes.pkl')
df_crimes.drop('index', inplace=True, axis=1)

# Import the Pickle of the Top Venues DataFrame
df_top_venues = pd.read_pickle('./capstone_pickles/top_venues.pkl')

# Import the Pickle of the Restaurants DataFrame
df_rest = pd.read_pickle('./capstone_pickles/restaurants.pkl')

## Top venues with more than 10 Restaurants nearby

In [140]:
# The Top Venues / Sites from the restaurant dataframe
top_venues_list = df_rest.venue_name.unique().tolist()

In [141]:
# Filter the top venues dataframe using the top venues list
df_top_venues = df_top_venues[df_top_venues['name'].isin(top_venues_list)]

# Take just the top 10 venues, sorted by score, to reduce to show the concept
df_top10_venues = df_top_venues.sort_values('score', ascending=False)[:10]

# Create a list of the top 10 venues names
top10_venues_list = df_top10_venues.name.tolist()

# Finally filter the Restaurants data frame to include only the top 10 venues
df_rest = df_rest[df_rest['venue_name'].isin(top10_venues_list)]
df_rest.reset_index(inplace=True)

In [143]:
# Create a list of the 2 most commonly occuring crimes
top_two_crimes = df_crimes[['primary_description', 'case']].groupby(
    ['primary_description']).count().sort_values('case', ascending=False)[:2].axes[0].tolist()

# Create a smaller DataFrame of only the top two crimes
df_crimes = df_crimes[df_crimes['primary_description'].isin(top_two_crimes)]
df_crimes.reset_index(inplace=True)
df_crimes.drop('index', inplace=True, axis=1)

In [161]:

df_dist = pd.DataFrame()

for name, lat, lon in zip(df_top10_venues.name,
                          df_top10_venues.latitude,
                          df_top10_venues.longitude):
    print('Processing: ', name)
    df_temp = df_crimes.copy()
    df_temp['ven_lat'] = lat
    df_temp['ven_lon'] = lon
    df_dist[name] = df_temp.apply(haversine, axis=1)
    
    


Processing:  Millennium Park
Processing:  Publican Quality Meats
Processing:  The Chicago Theatre
Processing:  Symphony Center (Chicago Symphony Orchestra)
Processing:  Grant Park
Processing:  Chicago Riverwalk
Processing:  Music Box Theatre
Processing:  Joe's Seafood, Prime Steak & Stone Crab
Processing:  The Art Institute of Chicago
Processing:  Unabridged Books


In [162]:
df_dist.head()

,Millennium Park,Publican Quality Meats,The Chicago Theatre,Symphony Center (Chicago Symphony Orchestra),Grant Park,Chicago Riverwalk,Music Box Theatre,"Joe's Seafood, Prime Steak & Stone Crab",The Art Institute of Chicago,Unabridged Books
0,11.308144,11.952016,11.635399,10.929078,10.275200,11.824240,19.081990,12.328596,10.970968,17.947193
1,13.079569,11.354375,12.640466,13.290260,13.987550,12.512433,6.218431,12.268728,13.319649,8.036216
2,1.556530,1.878425,1.170218,1.899576,2.613288,0.985704,6.617993,0.573648,1.878933,5.214137
3,1.777523,3.281721,2.166615,1.434160,0.717567,2.345192,9.914641,2.805724,1.449813,8.541500
4,12.969900,13.032318,13.206619,12.580159,12.044148,13.392086,19.989313,13.919421,12.644706,19.145570


In [163]:
df_crimes_01 = df_crimes.copy()
df_crimes_01['dist'] = df_dist['Millennium Park']
df_crimes_01 = df_crimes_01[df_crimes_01['dist'] <= 1.0]
df_crimes_01.shape

(6656, 27)

In [199]:
# Define Chicago's geolocation coordinates
chicago_latitude = df_top10_venues.latitude[df_top10_venues.name == 'Millennium Park'].values[0]  
chicago_longitude = df_top10_venues.longitude[df_top10_venues.name == 'Millennium Park'].values[0]

chicago_heatmat = folium.Map(location=[chicago_latitude, chicago_longitude], zoom_start=15) 

# List comprehension to make out list of lists
heat_data = [[row['latitude'], 
              row['longitude']] for index, row in df_crimes_01.iterrows()]

# Plot it on the map
HeatMap(heat_data,
        min_opacity=0.5,
        max_zoom=18, 
        max_val=1.0, 
        radius=15,
        blur=20,
        gradient=None,
        overlay=True).add_to(chicago_heatmat)

# Display the map
chicago_heatmat

In [200]:
df_crimes_02 = df_crimes.copy()
df_crimes_02['dist'] = df_dist['Publican Quality Meats']
df_crimes_02 = df_crimes_02[df_crimes_02['dist'] <= 1]
df_crimes_02.shape

(1534, 27)

In [201]:
# Define Chicago's geolocation coordinates
chicago_latitude = df_top10_venues.latitude[df_top10_venues.name == 'Publican Quality Meats'].values[0]  
chicago_longitude = df_top10_venues.longitude[df_top10_venues.name == 'Publican Quality Meats'].values[0]

chicago_heatmat = folium.Map(location=[chicago_latitude, chicago_longitude], zoom_start=15) 


# List comprehension to make out list of lists
heat_data = [[row['latitude'], 
              row['longitude']] for index, row in df_crimes_02.iterrows()]

# Plot it on the map
HeatMap(heat_data,
        min_opacity=0.5,
        max_zoom=18, 
        max_val=1.0, 
        radius=15,
        blur=20,
        gradient=None,
        overlay=True).add_to(chicago_heatmat)

# Display the map
chicago_heatmat

In [202]:
df_crimes_03 = df_crimes.copy()
df_crimes_03['dist'] = df_dist['The Chicago Theatre']
df_crimes_03 = df_crimes_03[df_crimes_03['dist'] <= 1.0]
df_crimes_03.shape

(8154, 27)

In [205]:
# Define Chicago's geolocation coordinates
chicago_latitude = df_top10_venues.latitude[df_top10_venues.name == 'The Chicago Theatre'].values[0]  
chicago_longitude = df_top10_venues.longitude[df_top10_venues.name == 'The Chicago Theatre'].values[0]

chicago_heatmat = folium.Map(location=[chicago_latitude, chicago_longitude], zoom_start=15) 


# List comprehension to make out list of lists
heat_data = [[row['latitude'], 
              row['longitude']] for index, row in df_crimes_03.iterrows()]

# Plot it on the map
HeatMap(heat_data,
        min_opacity=0.5,
        max_zoom=18, 
        max_val=1.0, 
        radius=15,
        blur=20,
        gradient=None,
        overlay=True).add_to(chicago_heatmat)

# Display the map
chicago_heatmat

In [206]:
df_crimes_04 = df_crimes.copy()
df_crimes_04['dist'] = df_dist['Symphony Center (Chicago Symphony Orchestra)']
df_crimes_04 = df_crimes_04[df_crimes_04['dist'] <= 1.0]
df_crimes_04.shape

(6033, 27)

In [207]:
# Define Chicago's geolocation coordinates
chicago_latitude = df_top10_venues.latitude[df_top10_venues.name == 'Symphony Center (Chicago Symphony Orchestra)'].values[0]  
chicago_longitude = df_top10_venues.longitude[df_top10_venues.name == 'Symphony Center (Chicago Symphony Orchestra)'].values[0]

chicago_heatmat = folium.Map(location=[chicago_latitude, chicago_longitude], zoom_start=15) 


# List comprehension to make out list of lists
heat_data = [[row['latitude'], 
              row['longitude']] for index, row in df_crimes_04.iterrows()]

# Plot it on the map
HeatMap(heat_data,
        min_opacity=0.5,
        max_zoom=18, 
        max_val=1.0, 
        radius=15,
        blur=20,
        gradient=None,
        overlay=True).add_to(chicago_heatmat)

# Display the map
chicago_heatmat

In [208]:
df_crimes_05 = df_crimes.copy()
df_crimes_05['dist'] = df_dist['Grant Park']
df_crimes_05 = df_crimes_05[df_crimes_05['dist'] <= 1.0]
df_crimes_05.shape

(2174, 27)

In [209]:
# Define Chicago's geolocation coordinates
chicago_latitude = df_top10_venues.latitude[df_top10_venues.name == 'Grant Park'].values[0]  
chicago_longitude = df_top10_venues.longitude[df_top10_venues.name == 'Grant Park'].values[0]

chicago_heatmat = folium.Map(location=[chicago_latitude, chicago_longitude], zoom_start=15) 


# List comprehension to make out list of lists
heat_data = [[row['latitude'], 
              row['longitude']] for index, row in df_crimes_05.iterrows()]

# Plot it on the map
HeatMap(heat_data,
        min_opacity=0.5,
        max_zoom=18, 
        max_val=1.0, 
        radius=15,
        blur=20,
        gradient=None,
        overlay=True).add_to(chicago_heatmat)

# Display the map
chicago_heatmat

In [210]:
df_crimes_06 = df_crimes.copy()
df_crimes_06['dist'] = df_dist['Chicago Riverwalk']
df_crimes_06 = df_crimes_06[df_crimes_06['dist'] <= 1.0]
df_crimes_06.shape

(8471, 27)

In [211]:
# Define Chicago's geolocation coordinates
chicago_latitude = df_top10_venues.latitude[df_top10_venues.name == 'Chicago Riverwalk'].values[0]  
chicago_longitude = df_top10_venues.longitude[df_top10_venues.name == 'Chicago Riverwalk'].values[0]

chicago_heatmat = folium.Map(location=[chicago_latitude, chicago_longitude], zoom_start=15) 


# List comprehension to make out list of lists
heat_data = [[row['latitude'], 
              row['longitude']] for index, row in df_crimes_06.iterrows()]

# Plot it on the map
HeatMap(heat_data,
        min_opacity=0.5,
        max_zoom=18, 
        max_val=1.0, 
        radius=15,
        blur=20,
        gradient=None,
        overlay=True).add_to(chicago_heatmat)

# Display the map
chicago_heatmat

In [212]:
df_crimes_07 = df_crimes.copy()
df_crimes_07['dist'] = df_dist['Music Box Theatre']
df_crimes_07 = df_crimes_07[df_crimes_07['dist'] <= 1.0]
df_crimes_07.shape

(817, 27)

In [213]:
# Define Chicago's geolocation coordinates
chicago_latitude = df_top10_venues.latitude[df_top10_venues.name == 'Music Box Theatre'].values[0]  
chicago_longitude = df_top10_venues.longitude[df_top10_venues.name == 'Music Box Theatre'].values[0]

chicago_heatmat = folium.Map(location=[chicago_latitude, chicago_longitude], zoom_start=15) 


# List comprehension to make out list of lists
heat_data = [[row['latitude'], 
              row['longitude']] for index, row in df_crimes_07.iterrows()]

# Plot it on the map
HeatMap(heat_data,
        min_opacity=0.5,
        max_zoom=18, 
        max_val=1.0, 
        radius=15,
        blur=20,
        gradient=None,
        overlay=True).add_to(chicago_heatmat)

# Display the map
chicago_heatmat

In [214]:
df_crimes_08 = df_crimes.copy()
df_crimes_08['dist'] = df_dist["Joe's Seafood, Prime Steak & Stone Crab"]
df_crimes_08 = df_crimes_08[df_crimes_08['dist'] <= 1.0]
df_crimes_08.shape

(7385, 27)

In [215]:
# Define Chicago's geolocation coordinates
chicago_latitude = df_top10_venues.latitude[df_top10_venues.name == "Joe's Seafood, Prime Steak & Stone Crab"].values[0]  
chicago_longitude = df_top10_venues.longitude[df_top10_venues.name == "Joe's Seafood, Prime Steak & Stone Crab"].values[0]

chicago_heatmat = folium.Map(location=[chicago_latitude, chicago_longitude], zoom_start=15) 


# List comprehension to make out list of lists
heat_data = [[row['latitude'], 
              row['longitude']] for index, row in df_crimes_08.iterrows()]

# Plot it on the map
HeatMap(heat_data,
        min_opacity=0.5,
        max_zoom=18, 
        max_val=1.0, 
        radius=15,
        blur=20,
        gradient=None,
        overlay=True).add_to(chicago_heatmat)

# Display the map
chicago_heatmat

In [216]:
df_crimes_09 = df_crimes.copy()
df_crimes_09['dist'] = df_dist['The Art Institute of Chicago']
df_crimes_09 = df_crimes_09[df_crimes_09['dist'] <= 1.0]
df_crimes_09.shape

(5941, 27)

In [217]:
# Define Chicago's geolocation coordinates
chicago_latitude = df_top10_venues.latitude[df_top10_venues.name == 'The Art Institute of Chicago'].values[0]  
chicago_longitude = df_top10_venues.longitude[df_top10_venues.name == 'The Art Institute of Chicago'].values[0]

chicago_heatmat = folium.Map(location=[chicago_latitude, chicago_longitude], zoom_start=15) 


# List comprehension to make out list of lists
heat_data = [[row['latitude'], 
              row['longitude']] for index, row in df_crimes_09.iterrows()]

# Plot it on the map
HeatMap(heat_data,
        min_opacity=0.5,
        max_zoom=18, 
        max_val=1.0, 
        radius=15,
        blur=20,
        gradient=None,
        overlay=True).add_to(chicago_heatmat)

# Display the map
chicago_heatmat

In [197]:
df_crimes_10 = df_crimes.copy()
df_crimes_10['dist'] = df_dist['Unabridged Books']
df_crimes_10 = df_crimes_10[df_crimes_10['dist'] <= 1.0]
df_crimes_10.shape

(1705, 27)

In [218]:
# Define Chicago's geolocation coordinates
chicago_latitude = df_top10_venues.latitude[df_top10_venues.name == 'Unabridged Books'].values[0]  
chicago_longitude = df_top10_venues.longitude[df_top10_venues.name == 'Unabridged Books'].values[0]

chicago_heatmat = folium.Map(location=[chicago_latitude, chicago_longitude], zoom_start=15) 

# List comprehension to make out list of lists
heat_data = [[row['latitude'], 
              row['longitude']] for index, row in df_crimes_10.iterrows()]

# Plot it on the map
HeatMap(heat_data,
        min_opacity=0.5,
        max_zoom=18, 
        max_val=1.0, 
        radius=15,
        blur=20,
        gradient=None,
        overlay=True).add_to(chicago_heatmat)

# Display the map
chicago_heatmat